# **Big Data Analytics Assessment** 
---
> Data Wrangling

> Run HiveQL queries for exploratory analysis
___

# 1. **Data Wrangling**

In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 836.5 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 453.9/644.9 MB 990.8 kB/s eta 0:03:13

In [81]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv("data/boneage-dataset.csv")

# Define image directory (assuming images are in the current directory and named by id)
image_dir = "."

# Image preprocessing parameters
img_size = 128

# Filter out rows where image file does not exist
df['image_path'] = df['id'].astype(str) + ".png"
df['image_path'] = df['image_path'].apply(lambda x: os.path.join(image_dir, x))
df = df[df['image_path'].apply(os.path.exists)]

# Load and preprocess images
images = []
labels = []

for _, row in df.iterrows():
    img = load_img(row['image_path'], target_size=(img_size, img_size), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0
    images.append(img_array)
    labels.append(row['boneage'])

X = np.array(images)
y = np.array(labels)

# Normalize labels
y = (y - y.mean()) / y.std()

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Regression output
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.2f}")

# Plot training history
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('Model MAE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.show()



ModuleNotFoundError: No module named 'tensorflow'